In [0]:
from pyspark.sql.functions import col

# Fake data creation (bronze layer)
# Customers data: id, first name, last name, and city
customer_data = [
    (1, "Mario",  "Rossi",   "Roma"),
    (2, "Luca",   "Bianchi", "Milano"),
    (3, "Giulia", "Verdi",   "Napoli"),
    (4, "Anna",   "Neri",    "Torino")
]

# Orders data: customer_id, order_id, amount
orders_data = [
    (1, 101, 250.5),
    (1, 102, 80.0),
    (2, 103, 300.0),
    (3, 104, 150.0),
    (3, 105, 20.0),
    (4, 106, 500.0),
    (4, 107, 45.0)
]

# Define schemas for DataFrame creation
customer_schema = "id INT, nome STRING, cognome STRING, citta STRING"
orders_schema = "customer_id INT, order_id INT, amount DOUBLE"

# Create DataFrames (bronze)
df_customers = spark.createDataFrame(customer_data, schema=customer_schema)
df_orders = spark.createDataFrame(orders_data, schema=orders_schema)

# Show raw data
print("Bronze layer - Customer data:")
df_customers.show()

print("Bronze layer - Orders data:")
df_orders.show()

Bronze layer - Customer data:
+---+------+-------+------+
| id|  nome|cognome| citta|
+---+------+-------+------+
|  1| Mario|  Rossi|  Roma|
|  2|  Luca|Bianchi|Milano|
|  3|Giulia|  Verdi|Napoli|
|  4|  Anna|   Neri|Torino|
+---+------+-------+------+

Bronze layer - Orders data:
+-----------+--------+------+
|customer_id|order_id|amount|
+-----------+--------+------+
|          1|     101| 250.5|
|          1|     102|  80.0|
|          2|     103| 300.0|
|          3|     104| 150.0|
|          3|     105|  20.0|
|          4|     106| 500.0|
|          4|     107|  45.0|
+-----------+--------+------+



In [0]:
df_customers.createOrReplaceTempView("customers_view")
df_orders.createOrReplaceTempView("orders_view")

silver_df_from_temp = spark.sql("""
    SELECT c.id AS customer_id, c.nome, c.cognome, c.citta, o.order_id, o.amount
    FROM customers_view c
    INNER JOIN orders_view o
        ON c.id = o.customer_id
    WHERE o.amount > 100
""")

print("Query From Temp Views:")
silver_df_from_temp.show()

Query From Temp Views:
+-----------+------+-------+------+--------+------+
|customer_id|  nome|cognome| citta|order_id|amount|
+-----------+------+-------+------+--------+------+
|          1| Mario|  Rossi|  Roma|     101| 250.5|
|          2|  Luca|Bianchi|Milano|     103| 300.0|
|          3|Giulia|  Verdi|Napoli|     104| 150.0|
|          4|  Anna|   Neri|Torino|     106| 500.0|
+-----------+------+-------+------+--------+------+



In [0]:
# Transformation into silver layer
silver_df = spark.sql(
    """
    SELECT
        c.id as customer_id,
        c.nome,
        c.cognome,
        c.citta,
        o.order_id,
        o.amount
    FROM {df_customers} c
    INNER JOIN {df_orders} o
        ON c.id = o.customer_id
    WHERE o.amount > 100
    """,
    df_customers=df_customers,
    df_orders=df_orders
)

print("Silver layer - Joined and filtered data (orders with amount > 100):")
silver_df.show()

# Further aggregation in the silver layer
silver_agg_df = spark.sql(
    """
    SELECT
        customer_id,
        nome,
        cognome,
        citta,
        SUM(amount) as total_amount,
        COUNT(order_id) as orders_count
    FROM {silver_df}
    GROUP BY customer_id, nome, cognome, citta
    """,
    silver_df=silver_df
)

print("Silver layer - Customer-level aggregation:")
silver_agg_df.show()

Silver layer - Joined and filtered data (orders with amount > 100):
+-----------+------+-------+------+--------+------+
|customer_id|  nome|cognome| citta|order_id|amount|
+-----------+------+-------+------+--------+------+
|          1| Mario|  Rossi|  Roma|     101| 250.5|
|          2|  Luca|Bianchi|Milano|     103| 300.0|
|          3|Giulia|  Verdi|Napoli|     104| 150.0|
|          4|  Anna|   Neri|Torino|     106| 500.0|
+-----------+------+-------+------+--------+------+

Silver layer - Customer-level aggregation:
+-----------+------+-------+------+------------+------------+
|customer_id|  nome|cognome| citta|total_amount|orders_count|
+-----------+------+-------+------+------------+------------+
|          1| Mario|  Rossi|  Roma|       250.5|           1|
|          2|  Luca|Bianchi|Milano|       300.0|           1|
|          3|Giulia|  Verdi|Napoli|       150.0|           1|
|          4|  Anna|   Neri|Torino|       500.0|           1|
+-----------+------+-------+------+---